<div align="left">
  <h1>Data Understanding</h1> <a name="0-bullet"></a>
</div>


---

In [ ]:
# import of the libraries for the exploration of the datasets
import numpy as np  
import pandas as pd

## Load the data

In [ ]:
# mounting the Drive folder
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Loading the two datasets
df_artists = pd.read_csv('/content/drive/MyDrive/Text_Analytics/Data/artists-data.csv') #load the list of artists
df_lyrics = pd.read_csv('/content/drive/MyDrive/Text_Analytics/Data/lyrics-data.csv') #load the list of songs

In [ ]:
# just take the columns "Link" and "Genre" from the artist dataset that will be used as link for the final one
df_artists_2c = df_artists[['Link', 'Genre']]
df_artists_2c.head()

,Link,Genre
0,/10000-maniacs/,Rock
1,/12-stones/,Rock
2,/311/,Rock
3,/4-non-blondes/,Rock
4,/a-cruz-esta-vazia/,Rock


## Data exploration

In [ ]:
# checking if there are duplicates
df_artists_2c.duplicated(subset = 'Link', keep = 'first').value_counts()

False    2940
True      302
dtype: int64

In [ ]:
# it seems that some artist has been classified with more than one genre
df_artists_2c.loc[lambda df: df['Link'] == '/10000-maniacs/']

,Link,Genre
0,/10000-maniacs/,Rock
1947,/10000-maniacs/,Pop


In [ ]:
# Which are the most used languages ?
df_lyrics.Idiom.value_counts()

ENGLISH           114723
PORTUGUESE         85085
SPANISH             4812
ITALIAN              626
FRENCH               471
GERMAN               314
KINYARWANDA           88
ICELANDIC             47
SWEDISH               27
FINNISH               24
INDONESIAN            17
ESTONIAN              12
GALICIAN              12
IRISH                  9
HAITIAN_CREOLE         9
DANISH                 9
BASQUE                 8
NORWEGIAN              7
TAGALOG                7
CROATIAN               7
CATALAN                6
SUNDANESE              6
DUTCH                  5
SWAHILI                5
MALAY                  4
RUSSIAN                4
SERBIAN                3
KURDISH                2
TURKISH                2
CEBUANO                2
NYANJA                 2
ARABIC                 2
JAPANESE               2
MALAGASY               2
SESOTHO                2
AFRIKAANS              1
BOSNIAN                1
KOREAN                 1
ROMANIAN               1
CZECH                  1


In [ ]:
# Just take the english songs
df_lyrics_en = df_lyrics.drop(df_lyrics[df_lyrics['Idiom'] !='ENGLISH'].index)
df_lyrics_en.Idiom.value_counts()

ENGLISH    114723
Name: Idiom, dtype: int64

In [ ]:
# check for duplicates in the lyrics dataset
df_lyrics_en.duplicated(subset = 'SLink', keep = 'first').value_counts()

False    91611
True     23112
dtype: int64

In [ ]:
# Drop duplicates in the field 'SLink'
df_lyrics_en.drop_duplicates(subset='SLink', keep='first', inplace=True, ignore_index=False)

# Search for persisting duplicate lyric entries:
df_lyrics_en.duplicated(subset = 'Lyric', keep = 'first').value_counts()

False    90796
True       815
dtype: int64

In [ ]:
# take only the columns used for the merge with the other dataset
df_lyrics_nd = df_lyrics_en.drop(['SName', 'SLink', 'Idiom'], axis=1)
df_lyrics_nd

,ALink,Lyric
0,/10000-maniacs/,I could feel at the time. There was no way of ...
1,/10000-maniacs/,"Take me now, baby, here as I am. Hold me close..."
2,/10000-maniacs/,These are. These are days you'll remember. Nev...
3,/10000-maniacs/,"A lie to say, ""O my mountain has coal veins an..."
4,/10000-maniacs/,Trudging slowly over wet sand. Back to the ben...
...,...,...
207611,/sambo/,"(Chorus). Hello, hello,hello,how low. Hello,he..."
207624,/sambo/,Well sometimes I go out by myself. And I look ...
207628,/sambo/,Feeling my way through the darkness. Guided by...
207792,/seu-jorge/,"Don't, don't, that's what you say. Each time t..."


In [ ]:
# Discard all duplicate rows: 
df_lyrics_nd.drop_duplicates(inplace=True)

# Check that we have absolutely no duplicate entries remaining:
df_lyrics_nd.duplicated().value_counts()

False    91392
dtype: int64

In [ ]:
# merge the two datasets
df_merged = pd.merge(df_lyrics_nd, df_artists_nd, how='inner', left_on='ALink', right_on='Link')
df_merged.head()

,ALink,Lyric,Link,Genre
0,/10000-maniacs/,I could feel at the time. There was no way of ...,/10000-maniacs/,Rock
1,/10000-maniacs/,"Take me now, baby, here as I am. Hold me close...",/10000-maniacs/,Rock
2,/10000-maniacs/,These are. These are days you'll remember. Nev...,/10000-maniacs/,Rock
3,/10000-maniacs/,"A lie to say, ""O my mountain has coal veins an...",/10000-maniacs/,Rock
4,/10000-maniacs/,Trudging slowly over wet sand. Back to the ben...,/10000-maniacs/,Rock


In [ ]:
# drop all the columns that we won't use
df_merged_2c = df_merged.drop(['ALink','Link'], axis=1)
df_merged_2c.head()

,Lyric,Genre
0,I could feel at the time. There was no way of ...,Rock
1,"Take me now, baby, here as I am. Hold me close...",Rock
2,These are. These are days you'll remember. Nev...,Rock
3,"A lie to say, ""O my mountain has coal veins an...",Rock
4,Trudging slowly over wet sand. Back to the ben...,Rock


In [ ]:
# It seems that the dataset is unbalanced
df_merged_2c.Genre.value_counts()

Rock            47534
Pop             25647
Hip Hop         13661
Sertanejo          51
Samba              42
Funk Carioca       15
Name: Genre, dtype: int64

In [ ]:
# so we will take only the first 3 genres
df_3g = df_merged_2c.drop(df_merged_2c[ (df_merged_2c['Genre'] == 'Sertanejo') | (df_merged_2c['Genre'] == 'Samba') | (df_merged_2c['Genre'] == 'Funk Carioca')].index)
df_3g

,Lyric,Genre
0,I could feel at the time. There was no way of ...,Rock
1,"Take me now, baby, here as I am. Hold me close...",Rock
2,These are. These are days you'll remember. Nev...,Rock
3,"A lie to say, ""O my mountain has coal veins an...",Rock
4,Trudging slowly over wet sand. Back to the ben...,Rock
...,...,...
86860,Smile though your heart. Is aching. Smile even...,Pop
86861,A dream like this. Not something. You wish for...,Pop
86862,"Aah, yeah yeah. I see the spotlight in my drea...",Pop
86863,"I'm, dreaming of a white, Christmas. Just like...",Pop


In [ ]:
df_3g.Genre.value_counts()

Rock       47534
Pop        25647
Hip Hop    13661
Name: Genre, dtype: int64

In [ ]:
# delete any eventual duplicate
df_3g.drop_duplicates(inplace=True)

In [ ]:
df_3g.head()

,Lyric,Genre
0,I could feel at the time. There was no way of ...,Rock
1,"Take me now, baby, here as I am. Hold me close...",Rock
2,These are. These are days you'll remember. Nev...,Rock
3,"A lie to say, ""O my mountain has coal veins an...",Rock
4,Trudging slowly over wet sand. Back to the ben...,Rock


In [ ]:
# # Save as .csv
# df_3g.to_csv('DF_3Genres_Lyrics_En.csv', index = False)